In [1]:
import sqlite3
import spacy
import preprocessing
import numpy as np

# 0. Fetch data from db

In [2]:
con = sqlite3.connect("../../db/db_training.sqlite")

cur = con.cursor()

In [3]:
# TODO: Use sqlutils.get_sql_statement()
with open("../../db/sql/select_all_text.sql") as sql_file:
    sql = sql_file.read()
    sql_file.close()

cur.execute(sql)
response = cur.fetchall()
'''
Response format: list[(text, category)]
'''

[(' 459e (nte assiSe premium WVec ( du Velour ctelle CANAPE ECO-PARTICIPATIONILCLuSE 3 PLACES 3598 CANAPE 2 PLACES Felataeeeed Existe aussi en 2598 ENFILADE 3 PORTES +1TIROIR FCo-DARICIPATLONINCIUSI 2798 TABLE EXTENSIBLE 160/200CM Bteadfm eoNVdns= 10908 CANAPE DANGLE AVANCE-RECULE Eco ParuIcIPAtonIccLuse Accoudoirs amovibles 300 3 dossiers avance-recule Existe aussi en CM Offre reservee aux titulaires du credit renouvelable Credit Modeme associe une cante credit Conforama Aurore facultative et gratuite, valable du 02/12/2023 au 30/12/2023 pour une PAIEMENT utilisation speciale de votre credit renouvelable de 600€  OOO€ (dans limite du montant disponible sur votre credit renouvelable) remboursable sur Une duree de ou 10,ou 10 mols_ La mensualitc de |\'utilisation speciale ajouto celle dos oporations courantes ovontuellcs ultoricurcs de votre credit renouvelablc Voir conditions de |\'offre aupres dc votre vendcur: x10 Sous reserve etude et d\'acceptation par Credit Modeme Ocean Indien SA

In [4]:
data_dict = {}
'''
Format: {[category]: list[page_text_content]}
'''

for page in response:
    text = page[0]
    category = page[1]

    if category not in list(data_dict.keys()):
        data_dict[category] = [text]
    else :
        data_dict[category].append(text)

print(len(data_dict["01"]))

{'01': [' 459e (nte assiSe premium WVec ( du Velour ctelle CANAPE ECO-PARTICIPATIONILCLuSE 3 PLACES 3598 CANAPE 2 PLACES Felataeeeed Existe aussi en 2598 ENFILADE 3 PORTES +1TIROIR FCo-DARICIPATLONINCIUSI 2798 TABLE EXTENSIBLE 160/200CM Bteadfm eoNVdns= 10908 CANAPE DANGLE AVANCE-RECULE Eco ParuIcIPAtonIccLuse Accoudoirs amovibles 300 3 dossiers avance-recule Existe aussi en CM Offre reservee aux titulaires du credit renouvelable Credit Modeme associe une cante credit Conforama Aurore facultative et gratuite, valable du 02/12/2023 au 30/12/2023 pour une PAIEMENT utilisation speciale de votre credit renouvelable de 600€  OOO€ (dans limite du montant disponible sur votre credit renouvelable) remboursable sur Une duree de ou 10,ou 10 mols_ La mensualitc de |\'utilisation speciale ajouto celle dos oporations courantes ovontuellcs ultoricurcs de votre credit renouvelablc Voir conditions de |\'offre aupres dc votre vendcur: x10 Sous reserve etude et d\'acceptation par Credit Modeme Ocean Ind

22


# 1. Cleaning

# 2. Create a vocabulary

# 3. Feature extraction (BOW)

TODO:
- Try auto-sklearn
- Create model.py (with pytorch)